In [9]:
import pandas as pd
from pathlib import Path
import re
import os

In [31]:
# === Paths ===
main_dir = "/mnt/d/night_recordings_analysis/"
validation_path = os.path.join(main_dir, "Barn Owl/brnowl_Validation.txt")
template_path = os.path.join(main_dir, "detection_history_template.csv")
output_path = os.path.join(main_dir, "Barn Owl/brnowl_detection_history.csv")

In [32]:
# Load template and fix index & column formatting
template = pd.read_csv(template_path)
template.columns = [str(col).strip() for col in template.columns]
template['site_id'] = template['site_id'].astype(str).str.strip().str.lower()
template.set_index('site_id', inplace=True)

# Ensure column names for survey nights are strings
template.columns = template.columns.astype(str)


In [33]:
# === Step 2: Load validation file ===
val_df = pd.read_csv(validation_path, sep="\t")

In [34]:
# === Step 3: Parse site and sn from filename like: 0.0059_54_m47_sn5.wav
def parse_site_sn(filename):
    match = re.search(r'_(ch|m|p)\d+_sn(\d+)', filename.lower())
    if match:
        site = match.group(0).split('_')[1]    # e.g., ch32
        sn = match.group(0).split('_')[2][2:]  # e.g., sn14 → 14
        return pd.Series([site.lower(), sn.lstrip("0")])  # site = lowercase, sn = "14"
    else:
        return pd.Series([None, None])

val_df[['site_id', 'SN']] = val_df['Begin File'].apply(parse_site_sn)

In [35]:
# Ensure types match
val_df['site_id'] = val_df['site_id'].astype(str).str.strip().str.lower()
val_df['SN'] = val_df['SN'].astype(str).str.strip()

In [36]:
# Filter only Eval == 1
val_df = val_df[val_df['Eval'] == 1]

In [37]:
# === Step 4: Update template ===
missing = []

for _, row in val_df.iterrows():
    site = row['site_id']
    sn = row['SN']
    try:
        if site in template.index and sn in template.columns:
            if pd.notna(template.loc[site, sn]):  # Don't overwrite blanks
                template.loc[site, sn] = 1
        else:
            missing.append((site, sn))
    except Exception as e:
        missing.append((site, sn))

# Optional: report missing
if missing:
    print("⚠️ The following site/SN pairs were not found in the template:")
    for site, sn in missing:
        print(f"  → site={site}, sn={sn}")
else:
    print("✅ All site/SN pairs successfully updated.")

# === Step 5: Save updated matrix
template.reset_index().to_csv(output_path, index=False)
print(f"✅ Detection history saved to: {output_path}")

✅ All site/SN pairs successfully updated.
✅ Detection history saved to: /mnt/d/night_recordings_analysis/Barn Owl/brnowl_detection_history.csv
